__Timing itertion alterntives__

In [ ]:
# time code in python: module time
import time
def timer (funct,*args):
    start = time.clock() # saves the system start time
    for i in range(1000):
        func(*args)
    return time.clock() - start # substract the finishing time with the start time

>>> from timer0 import timer # assuming that the function timer was stored in a different py file called timer0.py
>>> timer(pow, 2, 1000) # Time to call pow(2, 1000) 1000 times
0.00296260674205626
>>> timer(str.upper, 'spam') # Time to call 'spam'.upper() 1000 times
0.0005165746166859719

# however this is quite simple that is not reliable: Only windows, doesn't support keywords assignment, charges the timing cost of range(),etc

In [ ]:
# A more useful timer:
"""
Homegrown timing tools for function calls.
Does total time, best-of time, and best-of-totals time
"""

import time, sys
timer = time.clock if sys.platform[:3] == 'win' else time.time # time.clock is windows, otherwise time.time

def total(reps, func, *pargs, **kargs): # this one deals with differnt operative systmes and shows the time and result of the funct
    """
    Total time to run func() reps times.
    Returns (total time, last result)
    """
    repslist = list(range(reps)) # Hoist out, equalize 2.x, 3.x
    start = timer() # Or perf_counter/other in 3.3+ --> remember that it oes to time.clock or time.time
    for i in repslist:
        ret = func(*pargs, **kargs)
        elapsed = timer() - start
    return (elapsed, ret)

def bestof(reps, func, *pargs, **kargs):
    """
    Quickest func() among reps runs.
    Returns (best time, last result)
    """
    best = 2 ** 32 # 136 years seems large enough --> initial best timing --> the first iteration will beat this time for sure
    for i in range(reps): # range usage not timed here
        start = timer()
        ret = func(*pargs, **kargs)
        elapsed = timer() - start # Or call total() with reps=1
        if elapsed < best: best = elapsed # Or add to list and take min()
    return (best, ret)

def bestoftotal(reps1, reps2, func, *pargs, **kargs):
    """
    Best of totals:
    (best of reps1 runs of (total of reps2 runs of func))
    """
    return bestof(reps1, total, reps2, func, *pargs, **kargs) # uses reps1 in function bestof for using a funct reps 2 times -- merge bothe functions!!

>>> import timer
>>> timer.total(1000, pow, 2, 1000)[0] # Compare to timer0 results above
0.0029542985410557776
>>> timer.total(1000, str.upper, 'spam') # Returns (time, last call's result)
(0.000504845391709686, 'SPAM')
>>> timer.bestof(1000, str.upper, 'spam') # 1/1000 as long as total time
(4.887177027512735e-07, 'SPAM')
>>> timer.bestof(1000, pow, 2, 1000000)[0]
0.00393515497972885
>>> timer.bestof(50, timer.total, 1000, str.upper, 'spam') # the last two do basically the same !! --> remaind that function could be passed as objects
(0.0005468751145372153, (0.0005004469323637295, 'SPAM'))
>>> timer.bestoftotal(50, 1000, str.upper, 'spam')
(0.000566912540591602, (0.0005195069228989269, 'SPAM'))

# an alternative of best-of-all using generators :
>>> min(timer.total(1000, str.upper, 'spam') for i in range(50))
(0.0005155971812769167, 'SPAM')

__Newe timers calls in 3.3__
- __time.perf_counter()__ returns the value in fractional seconds of a performance
counter, defined as a clock with the highest available resolution to measure a short
duration. It includes time elapsed during sleep states and is system-wide.
- __time.process_time()__ returns the value in fractional seconds of the sum of the system
and user CPU time of the current process. It does not include time elapsed
during sleep, and is process-wide by definition.

In [ ]:
#use this ones instead of time.clock before defining the functions above
import time
if sys.version_info[0] >= 3 and sys.version_info[1] >= 3: # checking if the version of python is >= 3.3
timer = time.perf_counter # or process_time 
else:
timer = time.clock if sys.platform[:3] == 'win' else time.time

#or

try:
    timer = time.perf_counter # or process_time
except AttributeError:
    timer = time.clock if sys.platform[:3] == 'win' else time.time

In [ ]:
## code to compare different structures:
# File timeseqs.py
"Test the relative speed of iteration tool alternatives."
import sys, timer # Import timer functions
reps = 10000
repslist = list(range(reps)) # Hoist out, list in both 2.X/3.X
def forLoop():
    res = []
    for x in repslist:
        res.append(abs(x))
    return res

def listComp():
    return [abs(x) for x in repslist]

def mapCall():
    return list(map(abs, repslist)) # Use list() here in 3.X only! # return map(abs, repslist)

def genExpr():
    return list(abs(x) for x in repslist) # list() required to force results

def genFunc():
    def gen():
        for x in repslist:
            yield abs(x)
    return list(gen()) # list() required to force results

print(sys.version)
for test in (forLoop, listComp, mapCall, genExpr, genFunc):
    (bestof, (total, result)) = timer.bestoftotal(5, 1000, test)
    print ('%-9s: %.5f => [%s...%s]' % (test.__name__, bestof, result[0], result[-1]))

# results:
C:\code> c:\python33\python timeseqs.py
3.3.0 (v3.3.0:bd8afb90ebf2, Sep 29 2012, 10:57:17) [MSC v.1600 64 bit (AMD64)]
forLoop : 1.33290 => [0...9999]
listComp : 0.69658 => [0...9999]
mapCall : 0.56483 => [0...9999]
genExpr : 1.08457 => [0...9999]
genFunc : 1.07623 => [0...9999]

In [5]:
# you can use the module timeit in python
import timeit
'''which automates timing of code,
   supports command-line usage modes, and finesses some platform-specific issues'''
#use profile module (inlarger projects tha have to be timed) to isolate bottlenecks before recording the timing
import profile

__timeit library__ 

In [ ]:
import timeit
min(timeit.repeat(stmt="[x ** 2 for x in range(1000)]", number=1000, repeat=5))

'''repeat method returns a list with the total time testing a function
a number of times of each repeat: then with min we substract the min of them. In seconds'''

0.5062382371756811

In [ ]:
'''Some comments:'''
# Map wins the race if any function should be applied --> CPython terms
# The use of built-in function is faster then using user-defined functions
# list comprehension is quite faster than others tools in normal situations
#generators tend to be slower by a constant factor
# further information about code benchmarking:
 
 - pystone.py # C language benchmark program that was translated to Python by Python original creator Guido van Rossum
 - http://speed.python.org
 - http://speed.pypy.org


___FUNCTION COMMON GOTCHAS___

In [ ]:
# Local Names Are Detected Statically
X = 99
def selector(): # X used but not assigned --> look up in the LEGB rule
    print(X) # X found in global scope

selector()
99

# but what happen if you add an assignment after the reference:
def selector():
    print(X) # Does not yet exist! --> in the call, despite X global exists, the program will look for it in the locals!!
    X = 88 # X classified as a local name (everywhere) -->
           # Can also happen for "import X", "def X"...
selector()
UnboundLocalError: local variable 'X' referenced before assignment

'''The runing of def() det X as a local variable. Then, when the call(selector()) happens, Python look for X (to be printed)
   within the local variables. As X is assigned after the print, Python does not find it and raises an error. The LEGB rule
   applies when the def() is compilled !!'''

# possible solutions:

#1: state that X is global: However, it will change X value in al the module
def selector():
    global X # Force X to be global (everywhere)
    print(X)
    X = 88

>>> selector()
99

#2 import the working module (__main__) and print the X attribute. Then assign the local variable and print it.
X = 99
def selector():
    import __main__ # Import enclosing module
    print(__main__.X) # Qualify to get to global version of name
    X = 88 # Unqualified X classified as local
    print(X) # Prints local version of name
>>> selector()
99
88

In [ ]:
# Defaults and Mutable Objects --> retain state information

def saver(x=[]): # Saves away a list object --> mutable object !!! created at the def() runtime
    x.append(1) # Changes same object each time!
    print(x)
>>> saver([2]) # Default not used
[2, 1]
>>> saver() # Default used
[1]
>>> saver() # Grows on each call!
[1, 1]
>>> saver() # Doesnot get empty at each call
[1, 1, 1]

#Solution: make a copy or be sure to put the object in the body of the def --> renews the mutable object:
def saver(x=None):
    if x is None: # No argument passed? --> could be changed for: x = x or [], if x is empty, returns [].
        x = [] # Run code to make a new list each time
    x.append(1) # Changes new list object
    print(x)

>>> saver([2])
[2, 1]
>>> saver() # Doesn't grow here
[1]
>>> saver()
[1]

# Pythonic solution: function attributes --> much more explicit
def saver():
    saver.x.append(1)
    print(saver.x)

saver.x = [] # Set the attribute x as an empty list
>>> saver()
[1]
>>> saver()
[1, 1]
>>> saver()
[1, 1, 1]


In [ ]:
# Functions Without returns --> return None object automatically

def proc(x):
    print(x) # No return is a None return

x = proc('testing 123...') # x saves the return of proc() -> then, x = None
testing 123...
>>> print(x)
None

# can happen in common used methods:
>>> list = [1, 2, 3]
>>> list = list.append(4) # append is a "procedure" --> return nothing (None)
>>> print(list) # append changes list in place
None